In [1]:
%load_ext autoreload
%autoreload 2
import random
import html_parser
import ner
import psycopg2
import pandas as pd
import glob
from tqdm import tqdm
from names_parser import NameParser
import db
from features import map_article_to_size
tqdm.pandas()

2022-02-13 00:00:48,852 INFO __init__ Loading dictionaries from /home/tonko22/kukushka/venv/lib/python3.8/site-packages/pymorphy2_dicts_ru/data
2022-02-13 00:00:48,914 INFO __init__ format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [2]:
df = pd.read_pickle("df_single_article_preprocessed.pkl")
df.head(2)

,encode,case_doc_instance,case_document_articles,case_common_document_type,case_court_type,u_common_case_defendant_m,case_year_result,articles_extracted,articles_count
0,b09185e330815b9f0c858b33cf19ac5f,3,"[""Статья 228 Часть 1""]",Приговор,"Районный, городской, межрайонный суд",Габдуллин Р.Ф.. ... .01.09.2015. ... .ст.2...,2015,[Статья 228 Часть 1],1
1,8fcf5eb65bc203197f1bbef69ecfe732,3,"[""Статья 30 Часть 1, Статья 228.1 Часть 4 п.г""]",Приговор,"Районный, городской, межрайонный суд",Пятков П.А.. ... .01.09.2015. ... .ст. 30 ...,2015,"[Статья 30 Часть 1, Статья 228.1 Часть 4 п.г]",1


In [33]:
df['region'] = df['encode'].progress_apply(lambda x: db.fetch_region_db(uid=x)[0][0])

100%|██████████| 297040/297040 [49:27<00:00, 100.11it/s]


In [35]:
df['region'].value_counts()

23 Краснодарский край               18944
50 Московская область               15058
74 Челябинская область              11811
25 Приморский край                  11380
42 Кемеровская область - Кузбасс    10715
                                    ...  
83 Ненецкий автономный округ           52
79 Еврейская автономная область        29
95 Территории за пределами РФ           9
77 город Москва                         5
87 Чукотский автономный округ           3
Name: region, Length: 86, dtype: int64

In [36]:
df['region'].isna().value_counts()

False    296934
True        106
Name: region, dtype: int64

In [ ]:
text = db.load_text(doc_id)

In [60]:
df.to_pickle("df_single_article_preprocessed.pkl")

In [10]:
len(df)

297040

In [38]:
df['general_drug_amount'] = df['articles_extracted'].progress_apply(map_article_to_size)

100%|██████████| 297040/297040 [00:00<00:00, 479713.18it/s]


In [39]:
df['general_drug_amount'].isna().value_counts()

False    263329
True      33711
Name: general_drug_amount, dtype: int64

In [40]:
df['general_drug_amount'].value_counts()

Значительный     151329
Крупный          111364
Особо крупный       636
Name: general_drug_amount, dtype: int64

In [43]:
def conviction(uid: str):
    text = db.load_text(uid)
    if text is None:
        return None
    return ner.EntityExtractor.get_conviction(text)

def get_feature(uid: str, feature_method_name: str):
    text = db.load_text(uid)
    if text is None:
        return None
    try:
        method_to_call = getattr(ner.EntityExtractor, feature_method_name)
        return method_to_call(text)
    except BaseException as e:
        return None
    return method_to_call(text)

get_feature(uid="b09185e330815b9f0c858b33cf19ac5f",
           feature_method_name="get_conviction")

False

In [ ]:
import numpy as np
df['conviction'] = np.vectorize(get_feature)(df['encode'], "get_conviction")

In [ ]:
dfhead(2)

In [54]:
df['conviction'].value_counts()

False    154726
True      69182
Name: conviction, dtype: int64

In [55]:
df['conviction'].isna().value_counts()

False    223908
True      73132
Name: conviction, dtype: int64

In [46]:
df["conviction"] = df['encode'].progress_apply(get_feature, args=(["get_conviction"]))

100%|██████████| 297040/297040 [04:56<00:00, 1000.87it/s]


In [47]:
df["imprisonment"] = df['encode'].progress_apply(get_feature, args=(["get_imprisonment"]))

100%|██████████| 297040/297040 [08:53<00:00, 556.69it/s]


In [56]:
df["imprisonment"].isna().value_counts()

True     292072
False      4968
Name: imprisonment, dtype: int64

In [57]:
df["imprisonment"].value_counts()

True    4968
Name: imprisonment, dtype: int64

In [ ]:
df["punishment"] = df['encode'].progress_apply(get_feature, args=(["get_punishment"]))

In [58]:
df["drugs"] = df['encode'].progress_apply(get_feature, args=(["get_drugs"]))

2022-02-13 12:28:37,945 WARNING get_drugs No drugs found in whole text, check matches: []
 98%|█████████▊| 291610/297040 [2:13:33<03:06, 29.15it/s]2022-02-13 12:28:37,990 WARNING get_drugs No drugs found in whole text, check matches: []
2022-02-13 12:28:38,027 WARNING get_drugs No drugs found in whole text, check matches: [('ММ.ГГГГ около 21 часа 00 минут ФИО2, находясь по адресу: <адрес> в траве возле проезжей части между магазином «Пикник» и АЗС «Газпромнефть» забрал, тем самым незаконно приобрел вещество, без цели сбыта являющееся смесью, в состав которой входит синтетическое вещество «AB-PINACA-CHM» (МВА(N)-СНМ; [N-(1-Карбамоил-2-метилпропил)-1-циклогескилметил)-1Н-индазол -3-карбоксамид], которое является наркотическим средством – производным N-(1-карбамоил-2-метилпропил)-1-пентил-1Н-индазол-3-карбоксамида массой ', '19,89', 'гр.', ', то есть в крупном размере, которое он незаконно хранил при себе до задержания его сотрудниками полиции'), ('ММ.ГГГГ в 21 час 05 минут ФИО2 был задер

In [59]:
df["drugs"].isna().value_counts()

2022-02-13 12:31:58,056 INFO __init__ Loading dictionaries from /home/tonko22/kukushka/venv/lib/python3.8/site-packages/pymorphy2_dicts_ru/data
2022-02-13 12:31:58,124 INFO __init__ format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


False    170331
True     126709
Name: drugs, dtype: int64

In [33]:
import time

def extract_features(doc_id: str):
    text = db.load_text(doc_id)
    if text is None:
        return None
    
    feature_dict = ner.EntityExtractor.extract_features(text)
    feature_dict['sex'] = NameParser(text).sex
    feature_dict['doc_id'] = doc_id
    return feature_dict

#features = [extract_features(doc_id) for ]

2022-02-12 23:20:08,667 INFO __init__ Loading dictionaries from /home/tonko22/kukushka/venv/lib/python3.8/site-packages/pymorphy2_dicts_ru/data
2022-02-12 23:20:08,730 INFO __init__ format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [40]:
df_part = df.head(100)

In [41]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

results = [extract_features(doc_id) for doc_id in df_part.encode]

Could noto conver drug mass to float: .


In [43]:
df_part["Наркотики"] = [r["Наркотики"] for r in results]
df_part.head(2)

/tmp/ipykernel_884580/3989336699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part["Наркотики"] = [r["Наркотики"] for r in results]


,encode,case_doc_instance,case_document_articles,case_common_document_type,case_court_type,u_common_case_defendant_m,case_year_result,articles_extracted,articles_count,Наркотики
0,b09185e330815b9f0c858b33cf19ac5f,3,"[""Статья 228 Часть 1""]",Приговор,"Районный, городской, межрайонный суд",Габдуллин Р.Ф.. ... .01.09.2015. ... .ст.2...,2015,[Статья 228 Часть 1],1,None
1,8fcf5eb65bc203197f1bbef69ecfe732,3,"[""Статья 30 Часть 1, Статья 228.1 Часть 4 п.г""]",Приговор,"Районный, городской, межрайонный суд",Пятков П.А.. ... .01.09.2015. ... .ст. 30 ...,2015,"[Статья 30 Часть 1, Статья 228.1 Часть 4 п.г]",1,"амфетамин.?.?: 3,269 гр.; метилендиоксипировал..."


In [50]:
df_part["Наркотики"].isna().value_counts()

True     52
False    48
Name: Наркотики, dtype: int64

In [ ]:
df[~df["Наркотики"].isna() & df["Количество"].isna()][["Количество", "Наркотики"]]